In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_horizVert_wavelet_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,15087.28,-0.05,0.15,0.00,0.00,-0.01,0.00
3,-0.05,4058.25,-0.03,0.02,0.00,0.01,-0.00
4,0.15,-0.03,941.82,0.01,-0.00,0.00,-0.00
5,0.00,0.02,0.01,175.07,-0.00,-0.00,0.00
6,0.00,0.00,-0.00,-0.00,26.55,0.00,0.00
7,-0.01,0.01,0.00,-0.00,0.00,3.36,-0.00
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.27


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,-0.00001,0.00004,0.00000,0.00000,-0.00003,0.00000
3,-0.00001,1.00000,-0.00001,0.00003,0.00000,0.00005,-0.00002
4,0.00004,-0.00001,1.00000,0.00003,-0.00001,0.00002,-0.00000
5,0.00000,0.00003,0.00003,1.00000,-0.00001,-0.00000,0.00001
6,0.00000,0.00000,-0.00001,-0.00001,1.00000,0.00002,0.00003
7,-0.00003,0.00005,0.00002,-0.00000,0.00002,1.00000,-0.00001
8,0.00000,-0.00002,-0.00000,0.00001,0.00003,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.23790703678924946

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.25917616e+07 2.28795587e+06 1.18256869e+05 3.83263583e+03
 7.84387842e+01 9.45693872e-01 5.00061237e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998787,-0.048124,-0.010262,-0.001902,-0.000288,-0.000037,-0.000003
1,0.047543,0.997607,-0.049440,-0.008609,-0.001303,-0.000164,-0.000014
2,0.012488,0.048415,0.997548,-0.048459,-0.007042,-0.000890,-0.000072
3,0.002890,0.010756,0.047523,0.997436,-0.051950,-0.006385,-0.000517
4,0.000583,0.002167,0.009343,0.051019,0.996559,-0.064413,-0.005096
5,0.000111,0.000410,0.001771,0.009536,0.063366,0.994624,-0.081324
6,0.000018,0.000067,0.000289,0.001553,0.010238,0.080824,0.996675


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00121322, 0.00239311, 0.0024523 , 0.00256445, 0.00344107,
       0.005376  , 0.00332543])